In [9]:
DB_HOST = "localhost"
DB_PORT = "3306"
DB_USER = "root"
DB_PASSWORD = "urubu100"
DATABASE = "ALGAS"

CREATE_QUERY = "CREATE TABLE IF NOT EXISTS dados (id INT PRIMARY KEY AUTO_INCREMENT, grau varchar(15),turno varchar(31),mensalidade decimal(12,2),bolsa_integral_cotas int,bolsa_integral_ampla int,bolsa_parcial_cotas int,bolsa_parcial_ampla int,curso_busca varchar(127),uf_busca varchar(20),universidade_nome varchar(127),nota_integral_ampla decimal(12,2),nota_integral_cotas decimal(12,2));"
INSERT_QUERY = "INSERT INTO dados (id, grau, turno, mensalidade, bolsa_integral_cotas,bolsa_integral_ampla,bolsa_parcial_cotas,bolsa_parcial_ampla,curso_busca,uf_busca,universidade_nome,nota_integral_ampla,nota_integral_cotas) VALUES (DEFAULT, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
DELETE_QUERY = "TRUNCATE dados"

DROP_QUERY = "DROP TABLE dados"

In [4]:
from mysql.connector import connect as connect_my
def get_connection():
    return connect_my(
        # host = DB_HOST + ":" + DB_PORT,
        host = DB_HOST,
        user = DB_USER,
        password = DB_PASSWORD,
        database = DATABASE
        )

In [5]:
def get_insert_query():
    return INSERT_QUERY

def insert_data(self, data: list):
    con = get_connection()
    c = con.cursor()
    
    insert = get_insert_query()

    for value in data:
        c.execute(insert, value)

    con.commit()

def select_data():
    con = get_connection()
    c = con.cursor()
    
    c.execute(SELECT_QUERY)
    
    return c.fetchall()

In [6]:
class DatabaseManager:
    def __init__(self):
        self.connection = self.create()
        self.cursor = None
    
    def create(self):
        return get_connection()
    
    def transact_begin(self):
        self.cursor = self.connection.cursor()
    
    def transact_commit(self):
        self.connection.commit()

    def insert(self, dados: list):
        insert_data(self.connection, dados)
        
    def select(self) -> list:
        return select_data(cursor=self.connection.cursor())


In [7]:
from datetime import datetime
from sys import getsizeof
import os, json
import decimal
import random

with open('college_data.json', 'r', encoding='utf-8') as json_file:
    emps = json.load(json_file)

def get_random_decimal(i,d):
    return decimal.Decimal('%d.%d' % (random.randint(0,i),random.randint(0,d)))

def append_data(data) -> list:
    dados = []
    memory_accumulator = 0
    for range_data in data.values():
        for value in range_data:
            inicio = datetime.now()
            acumulador = 0
            for i in range(value, 0, -1):
                acumulador += i
            memory_accumulator += getsizeof(value)
            duracao = datetime.now() - inicio
            dados.append((value, memory_accumulator, duracao.microseconds))
    return dados

def generate_data(data : dict) -> list:
    dados = append_data(data)
    return dados 

def generate_data_random_college(rangeData) -> list:
    dataCollege = []
    for i in range(0,rangeData,1):
        dataGrau = random.choice(emps['grau'])
        dataTurno = random.choice(emps['turno'])
        dataMensalidade = get_random_decimal(9999,99)
        dataBolsaCota = random.randint(0,16)
        dataBolsaAmpla = random.randint(0,16)
        dataBolsaCotaParcial = random.randint(0,6)
        dataBolsaAmplaParcial = random.randint(0,10)
        dataCurso = random.choice(emps['curso_busca'])
        dataUF = random.choice(emps['uf_busca'])
        dataUniversidade = random.choice(emps['universidade_nome'])
        dataNotaAmpla = get_random_decimal(10**3,99)
        dataNotaCota = get_random_decimal(10**3,99)
        dataCollege.append([dataGrau,dataTurno,dataMensalidade,dataBolsaCota,dataBolsaAmpla,dataBolsaCotaParcial,dataBolsaAmplaParcial,dataCurso,dataUF,dataUniversidade,dataNotaAmpla,dataNotaCota])
    return dataCollege

In [8]:
rangeDadosInicio=0
rangeDadosFim=0
rangeDadosPulo=0
rangeDados={}
while True:
    print("Menu")
    print("1-Usuario, por favor insira range de dados 5 vezes:")
    print("2-sair")
    print("3-Inserir dados(somente depois do opcao 1)")
    print("4-Inserir dados randomicos escolares")
    answer = int(input("escolha:"))
    if answer == 2:
        break
    elif answer == 1:
        for i in range(0,5,1):
            print('range: ',i)
            rangeDadosInicio = int(input("insira o range inicio:"))
            rangeDadosFim = int(input("insira range fim:"))
            rangeDadosPulo = int(input("insira range pulo:"))
            rangeDados[i] = [rangeDadosInicio,rangeDadosFim,rangeDadosPulo]
        print('range: ', end='')
        doubleRange = {k:v for (k,v) in rangeDados.items()}
        print(doubleRange)
    elif answer == 3:        
        dadosX = generate_data(rangeDados)
        for i in dadosX:
            print('inserindo:',i)
        db = DatabaseManager()
        db.transact_begin()
        db.insert(dadosX)
        db.transact_commit()
        print("Inseriu")
    elif answer == 4:
        rangeEscola = int(input("Quantas vezes?"))
        dadosTotaisEscola = generate_data_random_college(rangeEscola)
        for i in dadosTotaisEscola:
            print('inserindo:',i)
        db = DatabaseManager()
        db.transact_begin()
        db.insert(dadosTotaisEscola)
        db.transact_commit()
        print("Inseriu")

Menu
1-Usuario, por favor insira range de dados 5 vezes:
2-sair
3-Inserir dados(somente depois do opcao 1)
4-Inserir dados randomicos escolares
escolha:4
Quantas vezes?500
inserindo: ['Tecnologo', 'Verpertino', Decimal('7755.67'), 5, 8, 6, 4, 'Gestão da Tecnologia da Informação', 'Rondônia', 'Faculdade de Ciências Econômicas, Administrativas e da Computação Dom Bosco - FCEACDB', Decimal('248.7'), Decimal('632.4')]
inserindo: ['Bacharelado', 'Verpertino', Decimal('6421.51'), 0, 8, 2, 10, 'Música - Cordas', 'Roraima', 'Faculdade Guaraí - FAG', Decimal('963.9'), Decimal('539.84')]
inserindo: ['Licenciatura', 'Verpertino', Decimal('1795.14'), 8, 16, 0, 6, 'Engenharia de Petróleo e Gás', 'Espírito Santo', 'Faculdade VIdal de Limoeiro - FAVILI', Decimal('823.54'), Decimal('189.21')]
inserindo: ['Bacharelado', 'Noturno', Decimal('9408.60'), 9, 6, 0, 6, 'Oceanografia', 'Rondônia', 'Faculdade América - América', Decimal('389.80'), Decimal('763.66')]
inserindo: ['Licenciatura', 'EAD', Decimal('3

Inseriu
Menu
1-Usuario, por favor insira range de dados 5 vezes:
2-sair
3-Inserir dados(somente depois do opcao 1)
4-Inserir dados randomicos escolares
escolha:4
Quantas vezes?200
inserindo: ['Tecnologo', 'Verpertino', Decimal('1202.48'), 14, 9, 3, 3, 'Letras - Inglês', 'Rio Grande do Norte', 'Universidade do Oeste Paulista - UNOESTE', Decimal('552.99'), Decimal('751.27')]
inserindo: ['Tecnologo', 'Integral', Decimal('9945.52'), 11, 12, 1, 7, 'Serviços Penais', 'Espírito Santo', 'Instituto de Ensino Superior de Mato Grosso - IESMT', Decimal('64.99'), Decimal('122.76')]
inserindo: ['Licenciatura', 'Integral', Decimal('6698.65'), 6, 2, 6, 7, 'Design de Moda', 'Mato Grosso do Sul', 'Faculdade de Ciências e Empreendedorismo - FACEMP', Decimal('435.63'), Decimal('590.35')]
inserindo: ['Bacharelado', 'Integral', Decimal('3285.32'), 9, 15, 5, 9, 'Eletrônica Industrial', 'Piauí', 'Escola Superior de Gestão Comercial e Marketing - ESIC', Decimal('410.45'), Decimal('691.91')]
inserindo: ['Tecnol

escolha:4
Quantas vezes?350
inserindo: ['Tecnologo', 'EAD', Decimal('7353.65'), 12, 14, 4, 8, 'Processos Escolares', 'Distrito Federal', 'Faculdade da Região Sisaleira - FARESI', Decimal('181.39'), Decimal('620.93')]
inserindo: ['Bacharelado', 'Verpertino', Decimal('1045.29'), 2, 10, 3, 5, 'Comunicação Social', 'Distrito Federal', 'Faculdade Una de Contagem - FUNAC', Decimal('130.80'), Decimal('691.36')]
inserindo: ['Bacharelado', 'Matutino', Decimal('5601.48'), 3, 2, 5, 7, 'Tradutor e Intérprete', 'Piauí', 'Faculdade Damas da Instrução Cristã - FADIC', Decimal('788.21'), Decimal('879.12')]
inserindo: ['Bacharelado', 'Integral', Decimal('8095.93'), 16, 4, 0, 7, 'Música - Canto', 'Alagoas', 'Faculdade de Agronegócios de Holambra - FAAGROH', Decimal('321.28'), Decimal('601.90')]
inserindo: ['Licenciatura', 'Noturno', Decimal('885.56'), 12, 4, 3, 2, 'Engenharia de Telecomunicações', 'Bahia', 'Faculdade Pio Décimo - FPD', Decimal('264.23'), Decimal('3.6')]
inserindo: ['Licenciatura', 'Verp

Inseriu
Menu
1-Usuario, por favor insira range de dados 5 vezes:
2-sair
3-Inserir dados(somente depois do opcao 1)
4-Inserir dados randomicos escolares
escolha:2
